In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 15 21:05:08 2018

@author: xshitova
"""

# -*- coding: utf-8 -*-
"""
Created on Fri Mar 16 12:16:55 2018

@author: xshitova
"""
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.cm as cm
from sklearn.naive_bayes import GaussianNB
from matplotlib.colors import Normalize

from sklearn import datasets, linear_model, metrics 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split 

dataset_test=pd.read_csv('C:/Users/xshitova/Desktop/anon/test_anon.csv', delimiter=';', keep_default_na=False)
dataset_train=pd.read_csv('C:/Users/xshitova/Desktop/anon/train1_anon.csv', delimiter=';',keep_default_na=False) 

cols=["feature1", "feature2", "feature3", "feature4", "feature5", 
      "feature6", "feature7", "feature8","feature9", 
      "feature10", "feature11", "feature12", "feature13"] 

product_1=["PRODUCT ONE"]
product_2=[ "PRODUCT TWO"]
product_3=["PRODUCT THREE"]
product_4=["PRODUCT FOUR"]
product_5=["PRODUCT FIVE"]
product_6=["PRODUCT SIX"]
product_7=["PRODUCT SEVEN"]

#-------------------Random tree on each product --------------------
print("")

dataset_test['feature1'].hist(bins=100)   # plotting histograms of all the features 
for i in range(0,12):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    plt.yscale('log')
    dataset_test[cols[i]].hist(bins=100)
    dataset_train[cols[i]].hist(bins=100, ax=ax, label='train', color='blue')
    #test and train are shown on the same graph 
    dataset_test[cols[i]].hist(bins=100, ax=ax, label='test', color='red')
    legend = ax.legend() 
    plt.title(cols[i])
    plt.yscale('log') #log scale of the axis 
    plt.xscale('log')

plt.matshow(dataset_train[cols].corr()) #the correlation matrix of features 
print(dataset_train[cols].corr()) #correlation values 

#plt.figure(figsize=(12, 16))
#t=np.arange(28994)
#plt.scatter(X_train["feature3"], X_train['feature10'], c=t) 
#plt.scatter(X_train["feature6"], X_train["feature3"])
#plt.xlabel('feature3')
#plt.xticks(X_train['feature5']) 
#plt.ylabel("feature1")
#plt.yticks(X_train['feature1'])
#plt.ylim(0,1000)
#plt.xlim(0,80)
#plt.show()

#these are unique values the categorical variables take and which will be turned into separate features 
feature4_unique=np.array([68,67,64,65,66,98,76,97,83,90,77,0,74,78,95,92,84,91,70])
feature2_unique=np.array([15,12,14,13,18,20,0,21,11,10,1,28,19,7,17,4,9,2,8,3,6,5])
feature6_unique=np.array([4228,4229,4236,4231,4230,4227,4226,4238,4233,4286,4239,4232,4241,4235,4240,4214,4225])
feature5_borders=np.array([192,383,409,506,551,655,771,853,873,950])

#this is cycles for creating new features for eachcategory of each categorical variable - here these are variables 2,4,5,6
#for i in range(0, len(X_train)):
for i in feature4_unique:
    cn='feature4_' + str(i)
    cols.append(cn)
    dataset_train[cn]=np.where(dataset_train['feature4']==i,1,0)
    dataset_test[cn]=np.where(dataset_test['feature4']==i,1,0)
#    if X_train['feature4'][i]=feature4_unique[0] segment_68='1' else '0'
    
for i in feature2_unique:
    cn='feature2_' + str(i)
    cols.append(cn)
    dataset_train[cn]=np.where(dataset_train['feature2']==i,1,0)
    dataset_test[cn]=np.where(dataset_test['feature2']==i,1,0)
    
for i in feature6_unique:
    cn='feature6_' + str(i)
    cols.append(cn)
    dataset_train[cn]=np.where(dataset_train['feature6']==i,1,0)
    dataset_test[cn]=np.where(dataset_test['feature6']==i,1,0)
    
j=0
prev=0
for i in feature5_borders:
    j=j+1
    cn='newfeature5_' + str(j)
    dataset_train[cn]=np.where((dataset_train['feature5']<=i)&(dataset_train['feature5']>=prev),j,0)
    dataset_test[cn]=np.where((dataset_test['feature5']<=i)&(dataset_test['feature5']>=prev),j,0)
    prev=i

dataset_train['newfeature5']=dataset_train['newfeature5_1']
dataset_train.drop(['newfeature5_1'], axis=1, inplace=True)
dataset_test['newfeature5']=dataset_test['newfeature5_1']
dataset_test.drop(['newfeature5_1'], axis=1, inplace=True)

for i in range(2,j+1):
    cn='newfeature5_' + str(i)  
    dataset_train['newfeature5'] = dataset_train['newfeature5'] + dataset_train[cn]
    dataset_test['newfeature5'] = dataset_test['newfeature5'] + dataset_test[cn]
    dataset_train.drop([cn], axis=1, inplace=True)
    dataset_test.drop([cn], axis=1, inplace=True)

#remove them as they've been categorized 
cols.append('newfeature5')
cols.remove('feature4')
cols.remove('feature1')
cols.remove('feature5')
cols.remove('feature11')
cols.remove('feature6')
cols.remove('feature2')

X_train=dataset_train[cols] #the explanatory variables train for 2016 - which sorts of clients bought these 7 products 
Y_train_product_2=dataset_train[product_2]
Y_train_product_3=dataset_train[product_3]
Y_train_product_5=dataset_train[product_5]
Y_train_product_6=dataset_train[product_6]
Y_train_product_7=dataset_train[product_7]

X_test=dataset_test[cols] #the explanatory variables test for 2017 - which sorts of clients are likely to buy these sorts of products 
Y_real_product_2=dataset_test[product_2]
Y_real_product_3=dataset_test[product_3]
Y_real_product_5=dataset_test[product_5]
Y_real_product_6=dataset_test[product_6]
Y_real_product_7=dataset_test[product_7]


#-------------------Random tree on each product --------------------
print("")
print("Random forest results on each one... 66 columns")
n3=200 #this is a variable the value os which can be changed or removed altogether - the number of estimators in a random forest model 

random_forest2 = RandomForestClassifier(n_estimators=n3)
random_forest2.fit(X_train, Y_train_product_2.values.ravel())
Y_pred_randomforest_2 = random_forest2.predict(X_test)
print("product 2 - " + str(metrics.accuracy_score(Y_real_product_2, Y_pred_randomforest_2)) )#model score product 2 

random_forest3 = RandomForestClassifier(n_estimators=n3)
random_forest3.fit(X_train, Y_train_product_3.values.ravel())
Y_pred_randomforest_3 = random_forest3.predict(X_test)
print("product 3 - "+str(metrics.accuracy_score(Y_real_product_3, Y_pred_randomforest_3))) #model score (so forth)

random_forest5 = RandomForestClassifier(n_estimators=n3)
random_forest5.fit(X_train, Y_train_product_5.values.ravel())
Y_pred_randomforest_5 = random_forest5.predict(X_test)
print("product 5 - "+str(metrics.accuracy_score(Y_real_product_5, Y_pred_randomforest_5))) #model score 

random_forest6 = RandomForestClassifier(n_estimators=n3)
random_forest6.fit(X_train, Y_train_product_6.values.ravel())
Y_pred_randomforest_6 = random_forest6.predict(X_test)
print("product 6 - "+str(metrics.accuracy_score(Y_real_product_6, Y_pred_randomforest_6))) #model score 

random_forest7 = RandomForestClassifier(n_estimators=n3)
random_forest7.fit(X_train, Y_train_product_7.values.ravel())
Y_pred_randomforest_7= random_forest7.predict(X_test)
print("product 7 - "+str(metrics.accuracy_score(Y_real_product_7, Y_pred_randomforest_7)) )#model score 

print("logistic regression...66 columns")

logreg6 = LogisticRegression()
logreg6.fit(X_train, Y_train_product_6.values.ravel())
Y_pred_logreg_6 = logreg6.predict(X_test)
print("product 6 - "+str(metrics.accuracy_score(Y_real_product_6, Y_pred_logreg_6)) )
